# This notebook will try to populate purchase and warranty info from HP, given an xlsx file containing exported inventory data

In [1]:
import pandas as pd
import xlrd
import xlwt
import requests
import simplejson as json
import time
import dateutil.parser
import datetime

In [2]:
#Super secret
apiKey='GWLESPEF2wHS5PwLFAUMzG9beA4LsF2Z'
apiSecret='zFlNL8kWkX2wpnb8oAxHbkNNn5Q39VAX'
tokenBody = { 'apiKey': apiKey, 'apiSecret': apiSecret, 'grantType': 'client_credentials', 'scope': 'warranty' }

### Parse exported data from inventory and collect data from "Serial" column

In [3]:
# Probably needs to be changed each time
inventory_cleaned_spreadsheet = 'copied.xlsx'

In [4]:
df = pd.read_excel(inventory_cleaned_spreadsheet)
print(df['Serial'])

0    MXL8021DPB
1    2UA8141LLQ
Name: Serial, dtype: object


In [5]:
data = []
for row in df.Serial.iteritems():
    temp_dict = {'sn': row[1]}
    #print(temp_dict)
    data.append(temp_dict)
print(data)

[{'sn': 'MXL8021DPB'}, {'sn': '2UA8141LLQ'}]


In [6]:
def _url(path):
    return 'https://css.api.hp.com' + path

In [7]:
tokenHeaders = { 'Accept': 'application/json' }
tokenResponse = requests.post(_url('/oauth/v1/token'), data=tokenBody, headers=tokenHeaders)
tokenJson = tokenResponse.json()
print(tokenJson)
token = tokenJson['access_token']
jobHeaders = {
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + token,
    'Content-Type': 'application/json'
}
#print(jobHeaders)

{'access_token': 'GAYYbLEIihNDdJBuq7GuifobUXhF', 'expires_in': '3599', 'scope': 'warranty'}


### Run the API calls with HP

In [10]:
print('Creating new batch job...')
jobResponse = requests.post(_url('/productWarranty/v1/jobs/'), data = json.dumps(data), headers = jobHeaders)
job = jobResponse.json()

print('Batch job created successfully.')
print('--------------------')
print(job)
print('Job ID: ' + job['jobId'])
print('Estimated time in seconds to completion: ' + str(job['estimatedTime']))
print('')
if (job['estimatedTime'] > 1200):
    time.sleep(600)
else:
    time.sleep(300)

Creating new batch job...
Batch job created successfully.
--------------------
{'fault': {'faultstring': 'Unable to identify proxy for host: CSSITSecure and url: /productWarranty/v1/jobs/', 'detail': {'errorcode': 'messaging.adaptors.http.flow.ApplicationNotFound'}}}


KeyError: 'jobId'

Waiting around for it to finish

In [ ]:
headers = {
    'Authorization': 'Bearer ' + token,
    'Accept-Encoding': 'gzip,deflate'
}
status = 'incomplete'

while (status == 'incomplete'):
    monitorResponse = requests.get('https://css.api.hp.com/productWarranty/v1/jobs/' + job['jobId']), headers=headers)
    monitor = monitorResponse.json()
    if (monitor['status'] != "completed"):
        if (monitor['estimatedTime'] > 1200):
            print('Estimated time in seconds to completion: ' + str(monitor['estimatedTime']) + '\nNext job check in 10 minutes...\n')
            time.sleep(200)
        elif (monitor['estimatedTime'] > 600):
            print('Estimated time in seconds to completion: ' + str(monitor['estimatedTime']) + '\nNext job check in 5 minutes...\n')
            time.sleep(100)
        else:
            print('Estimated time in seconds to completion: ' + str(monitor['estimatedTime']) + '\nNext job check in 1 minute...\n')
            time.sleep(10)
    else:
        status = 'complete'

### Print out the purchase and warranty dates

In [ ]:
resultsResponse = requests.get(_url('/productWarranty/v2/jobs/' + job['jobId'] + '/results'), headers=headers)
results = resultsResponse.json()
today = datetime.date.today()

for r in results:
    serialNumber = r["product"]["serialNumber"]
    print("\n=====New Product=====")
    for offer in r["offers"]:
        if "HP HW Maintenance Onsite Support" in offer["offerDescription"]:
            print("Warranty Started: " + offer["serviceObligationLineItemStartDate"])
            print("Warranty Ended: " + offer["serviceObligationLineItemEndDate"])
            parsed = dateutil.parser.parse(offer['serviceObligationLineItemEndDate']).date()
            if today < parsed:
                print('Warranty active for ' + serialNumber + "\n")
            else:
                print('Warranty inactive for ' + serialNumber + "\n")

### Save a raw json file with the data

In [ ]:
try:
    f = open(job['jobId'] + '.json', 'w')
    f.write(json.dumps(results))
    print("\nWarranty information was retrieved for " + str(len(results)) + " objects.\nTo view raw data, see " + job['jobId'] + ".json.")
    f.close()
except Exception:
    print("\nWarranty information was retrieved for " + str(len(results)) + ' objects.\nRaw data could not be written to file.')